# Polynomial Regression - Gradient Descent Model Selection

In this notebook, we show how to perform model selection when using Stochastic Gradient Descent (SGD) algorithm for a Polynomial Regression Model. There are two sets of hyperparameters for SGD based Polynomial Regression model.

- Polynomial degree
- SGD algorithm hyperparameters (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html)


## Data Transformation and Model Creation

The SGD Polynomial Regressor model requires the data to be augmented by the following two transformations.
- Create polynomial features based on the degree of the polynomial
- Standardize the augmented dataset

Then, a SGD Regressor model is created, which uses the augmented data for training.

For model selection, we need to create several models based on the polynomial degree as well as combinations of a range of values of the SGD Regressor hyperparameters.

The above two data transformation steps can be combined in a single model step by using Scikit-Learn's Pipeline object.
https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

The Pipeline object will assemble several steps that can be cross-validated together while setting different parameters. More specifically it will perform the following three steps to create a SGD Polynomial Regression model.

- Create polynomial features based on the degree of the polynomial
- Standardize the augmented dataset
- Create a SGD Regressor model, which will use the standardized data for training



## Dataset

We use the Boston housing dataset that provides housing values in the suburbs of Boston.

URL: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html#sklearn.datasets.load_boston


The **MEDV** variable is the target variable.

### Data description

The Boston data frame has 506 rows and 14 columns.

This data frame contains the following columns:

- CRIM: per capita crime rate by town.

- ZN: proportion of residential land zoned for lots over 25,000 sq.ft.

- INDUS: proportion of non-retail business acres per town.

- CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).

- NOX: nitrogen oxides concentration (parts per 10 million).

- RM: average number of rooms per dwelling.

- AGE: proportion of owner-occupied units built prior to 1940.

- DIS: weighted mean of distances to five Boston employment centres.

- RAD: index of accessibility to radial highways.

- TAX: full-value property-tax rate per $10,000.

- PTRATIO: pupil-teacher ratio by town.

- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.

- LSTAT: lower status of the population (percent).

- MEDV: median value of owner-occupied homes in $1000s.

In [1]:
import warnings
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

## Load Data

First load the data and explore the feature names, target names, etc.

In [2]:
# Load data
boston = datasets.load_boston()
print(boston.data.shape, boston.target.shape)
print(boston.feature_names)

(506, 13) (506,)
['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


## Create A DataFrame Object

In [3]:
df = pd.DataFrame(boston.data,columns=boston.feature_names)
df = pd.concat([df, pd.Series(boston.target,name='MEDV')], axis=1)

## Create a Separate Feature Set (Data Matrix X) and Target (1D Array y)

Create a data matrix (X) that contains all features and a 1D target array (y) containing the target.

First, we create separate data frame objects for X and y. Then, we convert the data frame objects into arrays.

In [4]:
# Create separate data frame objects for X (features) and y (target)
X = df.drop(columns='MEDV')  
y = df['MEDV'] 


X = np.asarray(X) # Data Matrix containing all features excluding the target
y = np.asarray(y) # 1D target array


print("Data Matrix (X) Shape: ", X.shape)
print("Label Array (y) Shape: ", y.shape)

print("\nData Matrix (X) Type: ", X.dtype)
print("Label Array (y) Type: ", y.dtype)

Data Matrix (X) Shape:  (506, 13)
Label Array (y) Shape:  (506,)

Data Matrix (X) Type:  float64
Label Array (y) Type:  float64


## Create Train and Test Dataset

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Model Selection

For hyperparameter tuning, we build a compund regressor using the Scikit-Learn’s  Pipeline class. It will combine the PolynomialFeatures(), StandardScaler() and SGDRegressor() objects and will create a single object.

The best way to do hyperparameter tuning is to use cross-validation. We will use Scikit-Learn’s GridSearchCV to search the combinations of hyperparameter values that provide best performance.

We need to tell which hyperparameters we want the GridSearchCV to experiment with, and what values to try out. It will evaluate all the possible combinations of hyperparameter values, using cross-validation.

To denote the module objects in the Pipeline we use arbitrary names: "poly", "scaler" and "sgd". We use these names to perform grid search for the respective optimal hyperparameters.

In [6]:
%%time
warnings.filterwarnings('ignore')

# Create a Pipeline object
sgd_pipeline = Pipeline([
        # Bias should be excluded because by default SGDRegressor adds bias via the"fit_intercept" parameter
        ('poly', PolynomialFeatures(include_bias=False)), 
        ('scaler', StandardScaler()),
        ('sgd', SGDRegressor(penalty='elasticnet')),
    ])

# Create a dictionary object with hyperparameters as keys and lists of corresponding values
param_grid = {'poly__degree': [1, 2, 3, 4, 5],
              'sgd__alpha': [0.1, 0.01, 0.001, 0.0001], 
              'sgd__l1_ratio': [1, 0.7, 0.5, 0.2, 0], 'sgd__max_iter':[500, 1000],
              'sgd__eta0': [0.01, 0.001, 0.0001]}

# Create a GridSearchCV object and perform hyperparameter tuning
sgd = GridSearchCV(sgd_pipeline, param_grid, scoring='neg_mean_squared_error', cv=3, verbose=1, n_jobs=-1)

# The model is trained with optimal hyperparameters, thus its the optimal model
sgd.fit(X_train, y_train)

# Get the optimal hyperparameters
params_optimal_sgd = sgd.best_params_

print("Best Score (negative mean squared error): %f" % sgd.best_score_)
print("Optimal Hyperparameter Values: ", params_optimal_sgd)
print("\n")

Fitting 3 folds for each of 600 candidates, totalling 1800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 1068 tasks      | elapsed:   37.5s
[Parallel(n_jobs=-1)]: Done 1341 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1691 tasks      | elapsed:  4.6min


Best Score (negative mean squared error): -15.580378
Optimal Hyperparameter Values:  {'poly__degree': 2, 'sgd__alpha': 0.001, 'sgd__eta0': 0.01, 'sgd__l1_ratio': 0.5, 'sgd__max_iter': 1000}


CPU times: user 4.87 s, sys: 407 ms, total: 5.27 s
Wall time: 5min 20s


[Parallel(n_jobs=-1)]: Done 1800 out of 1800 | elapsed:  5.3min finished


## Model Evaluation

We use the **optimal model created above** for its evaluation based on training and test data. There is no need to train the model again by using the optimal hyperparameters.

Note that the model is a Pipeline object. Before making predictions, it will transform the data by applying the optimal degree and standardization. Thus, we don't need to separately add polynomial terms and perform standardization for making predictions.

In [7]:
# Training data: Make prediction 
y_train_predicted_sgd = sgd.predict(X_train)

print("Train: Mean squared error: %.2f"
      % mean_squared_error(y_train, y_train_predicted_sgd))

# Training data: Explained variance score: 1 is perfect prediction
print("Train: Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" 
      % r2_score(y_train, y_train_predicted_sgd))

# Test data: Make prediction 
y_test_predicted = sgd.predict(X_test)

print("\nTest: Mean squared error: %.2f"
      % mean_squared_error(y_test, y_test_predicted))

# Training data: Explained variance score: 1 is perfect prediction
print("Test: Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" 
      % r2_score(y_test, y_test_predicted))

Train: Mean squared error: 11.40
Train: Coefficient of determination r^2 variance score [1 is perfect prediction]: 0.87

Test: Mean squared error: 13.19
Test: Coefficient of determination r^2 variance score [1 is perfect prediction]: 0.82
